In [78]:
#!pip3.9 install -U numpy
#!pip3.9 install --pre -U pycaret --user
# print("PyCaret Installed")

In [79]:
import numpy as np
import pandas as pd

In [80]:
tweets = pd.read_csv("cyberbullying_tweets.csv")
tweets

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [81]:
tweets["cyberbullying_type"].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [82]:
tweets20 = tweets.head(20000)

In [83]:
from pycaret.nlp import *
from pycaret.classification import *
from sklearn.feature_extraction.text import CountVectorizer

In [84]:
tf_vectorizer = CountVectorizer(min_df=.015, max_df=.8, max_features=5, ngram_range=[1, 3])

In [85]:
%time features = tf_vectorizer.fit_transform(tweets20['tweet_text'])
features

CPU times: user 3.64 s, sys: 87.6 ms, total: 3.73 s
Wall time: 3.81 s


<20000x5 sparse matrix of type '<class 'numpy.int64'>'
	with 29717 stored elements in Compressed Sparse Row format>

In [86]:
features_df = pd.DataFrame(features.toarray(), columns=tf_vectorizer.get_feature_names())
features_df

,and,is,the,to,you
0,0,0,0,0,0
1,0,1,0,0,0
2,0,0,0,0,0
3,0,0,1,0,0
4,0,2,0,1,0
...,...,...,...,...,...
19995,1,0,0,1,0
19996,0,0,1,0,0
19997,0,2,2,0,2
19998,0,0,1,0,1


In [87]:
df = pd.concat([features_df,tweets20['cyberbullying_type']],axis=1)
df

,and,is,the,to,you,cyberbullying_type
0,0,0,0,0,0,not_cyberbullying
1,0,1,0,0,0,not_cyberbullying
2,0,0,0,0,0,not_cyberbullying
3,0,0,1,0,0,not_cyberbullying
4,0,2,0,1,0,not_cyberbullying
...,...,...,...,...,...,...
19995,1,0,0,1,0,religion
19996,0,0,1,0,0,religion
19997,0,2,2,0,2,religion
19998,0,0,1,0,1,religion


In [88]:
#Shuffle your dataset 
shuffle_df = df.sample(frac=1)

# Define a size for your train set 
train_size = int(0.9 * len(df))

# Split your dataset 
train_df = shuffle_df[:train_size]
test_df  = shuffle_df[train_size:]

In [89]:
numerical_features = list(features_df.columns)
%time temp = setup(data = train_df, target = 'cyberbullying_type',numeric_features=numerical_features)

,Description,Value
0,Session id,487
1,Target,cyberbullying_type
2,Target type,Multiclass
3,Target mapping,"gender: 0, not_cyberbullying: 1, religion: 2"
4,Original data shape,"(18000, 6)"
5,Transformed data shape,"(18000, 6)"
6,Transformed train set shape,"(12599, 6)"
7,Transformed test set shape,"(5401, 6)"
8,Numeric features,5
9,Preprocess,True


CPU times: user 1.37 s, sys: 68.6 ms, total: 1.44 s
Wall time: 1.54 s


In [90]:
%time lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5087,0.6459,0.5087,0.5089,0.4882,0.2152,0.2269
1,0.5254,0.6765,0.5254,0.5355,0.5111,0.2435,0.2555
2,0.5159,0.6796,0.5159,0.5093,0.4999,0.2356,0.2434
3,0.5484,0.6995,0.5484,0.5503,0.5301,0.2835,0.2971
4,0.4865,0.6518,0.4865,0.4773,0.4630,0.1869,0.1971
5,0.5198,0.6751,0.5198,0.5219,0.5023,0.2363,0.2477
6,0.5175,0.6737,0.5175,0.5097,0.5008,0.2398,0.2478
7,0.5000,0.6651,0.5000,0.4933,0.4794,0.2039,0.2133
8,0.5206,0.6696,0.5206,0.5280,0.5003,0.2374,0.2528


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 2.61 s, sys: 351 ms, total: 2.96 s
Wall time: 11.2 s


In [91]:
%time tuned_lightgbm = tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5103,0.6532,0.5103,0.5082,0.4928,0.2199,0.2292
1,0.5341,0.6851,0.5341,0.5413,0.5219,0.2596,0.2698
2,0.5135,0.6810,0.5135,0.5064,0.4981,0.2328,0.2400
3,0.5556,0.6964,0.5556,0.5573,0.5407,0.2946,0.3057
4,0.4952,0.6524,0.4952,0.4887,0.4749,0.1991,0.2086
5,0.5183,0.6733,0.5183,0.5210,0.5028,0.2322,0.2418
6,0.5262,0.6777,0.5262,0.5197,0.5113,0.2562,0.2639
7,0.5151,0.6725,0.5151,0.5147,0.4991,0.2290,0.2383
8,0.5365,0.6706,0.5365,0.5486,0.5166,0.2621,0.2799


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
CPU times: user 5.79 s, sys: 244 ms, total: 6.03 s
Wall time: 28.1 s


In [106]:
#evaluate model
# predict_model(tuned_lightgbm)
predict_model(tuned_lightgbm,data=test_df)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.5200,0.6855,0,0,0,0.2449,0.2545


,and,is,the,to,you,cyberbullying_type,prediction_label,prediction_score
0,1.0,0.0,1.0,2.0,0.0,gender,gender,0.4879
1,0.0,1.0,0.0,0.0,0.0,not_cyberbullying,not_cyberbullying,0.4963
2,0.0,0.0,0.0,0.0,0.0,not_cyberbullying,not_cyberbullying,0.5922
3,1.0,0.0,2.0,0.0,2.0,gender,religion,0.5749
4,0.0,0.0,1.0,4.0,0.0,gender,religion,0.4232
...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,not_cyberbullying,not_cyberbullying,0.5922
1996,0.0,0.0,0.0,0.0,0.0,not_cyberbullying,not_cyberbullying,0.5922
1997,0.0,0.0,0.0,2.0,6.0,gender,religion,0.5349
1998,0.0,3.0,0.0,0.0,0.0,religion,gender,0.4483


In [97]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.5208,0.6731,0.5208,0.5190,0.5055,0.2401,0.2489,1.1050
ada,Ada Boost Classifier,0.5161,0.6556,0.5161,0.5128,0.5039,0.2328,0.2388,0.2370
lightgbm,Light Gradient Boosting Machine,0.5153,0.6695,0.5153,0.5136,0.4962,0.2305,0.2416,0.3090
lda,Linear Discriminant Analysis,0.5117,0.6669,0.5117,0.5078,0.5000,0.2247,0.2299,0.0680
ridge,Ridge Classifier,0.5105,0.0000,0.5105,0.5091,0.4898,0.2204,0.2322,0.0580
lr,Logistic Regression,0.5103,0.6658,0.5103,0.5117,0.5007,0.2163,0.2201,0.0960
rf,Random Forest Classifier,0.5054,0.6608,0.5054,0.5022,0.4873,0.2126,0.2217,0.3950
et,Extra Trees Classifier,0.5046,0.6572,0.5046,0.5032,0.4873,0.2079,0.2161,0.3560
dt,Decision Tree Classifier,0.5017,0.6517,0.5017,0.4997,0.4838,0.2020,0.2101,0.0610
qda,Quadratic Discriminant Analysis,0.4938,0.6553,0.4938,0.4801,0.4535,0.2031,0.2236,0.0620


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=487, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)